In [0]:
#from openai import AzureOpenAI
from datetime import datetime
import time
import shutil
import re
from collections import OrderedDict
import sys
import time
import os
from pathlib import Path
from pyspark.sql import Row


In [0]:

files = os.listdir("queries/")
subdir = "queries/"
SQL_FILE = subdir + files[0]
intab = files[0].lower().replace('.sql','')
with open(SQL_FILE, "r", encoding="utf-8") as files:
    my_sql = files.read()
prompt = my_sql
RowList = [Row(col) for col in prompt.replace('"/BIC/','').replace('"','').split(';') if col.replace('\n', '').replace(" ", "") != '' and ('ENDMETHOD') not in col]; 
RowList = spark.createDataFrame(RowList).withColumnRenamed("_1", "table_statement")
from pyspark.sql.functions import split, col, explode
RowList = RowList.withColumn("table_statement", explode(split(col("table_statement"), "---")))
ase = prompt.replace('\nOUTTAB', '').split(' ')
ase = set([x.replace(':',"").replace('\n','') for x in ase if x.startswith(':IT')])
RowList = RowList.filter((col("table_statement").rlike('|'.join(ase))) & (~col("table_statement").contains("OUTTAB")))

RowList = RowList.filter(~col("table_statement").contains("ENDMETHOD"))
start_index = [RowList.collect()[x][0].replace('.\nIT','.\n IT').replace('\nIT','\n IT').replace("\n   IT_", '\n IT_').find("\n IT_")+ 2 for x in range(len(RowList.collect()))]
RowList = [RowList.collect()[x][0].replace('.\nIT','.\n IT').replace('\nIT','\n IT').replace("\n   IT_", '\n IT_')[start_index[x]:].replace('\nIT','IT').replace('\n IT','IT').replace('\n  IT','IT').lower() for x in range(len(RowList.collect()))]
RowList = ['create or replace temp view '+ x.replace('=\n', 'as (\n').replace(':','').replace('intab', intab) + '\n)' for x in RowList]
mapping_dict = {}
for x in RowList:
    start_index = x.find(" as (\n") + len(" as (\n")
    x = x[start_index:]
    if 'z2' in x or 'z9' in x:
        z2_count = x.count('z2') + x.count('z9')
        if 'join' not in x:
            if 'union' in x:
                x_list = x.split('union')
                for y in x_list:
                    if 'z2' in y or 'z9' in y:
                        start_index = y.find("select ") + len("select ")
                        end_index = y.find("from")
                        column = y[start_index:end_index].replace('  ', '')
                        column_list = [item.split('--')[0].strip().split(' as ')[0].split(',')[0].split('(')[-1].split(')')[0].replace('=','').replace('>','').replace('<','') for item in column.split('\n') if item.split('--')[0].strip() != '']
                        from_name = y[end_index + len('from '):]
                        end_index_from = from_name.find('\n')
                        from_name = from_name[:end_index_from]
                        mapping_dict[from_name] = mapping_dict.get(from_name, set()) | set(column_list)
            else:
                start_index = x.find("select ") + len("select ")
                end_index = x.find("from")
                column = x[start_index:end_index].replace('  ', '')
                column_list = [item.replace('=','').replace('>','').replace('<','') for item in column.split('\n') if item != '']
                from_name = x[end_index + len('from '):]
                end_index_from = from_name.find('\n')
                from_name = from_name[:end_index_from]
                mapping_dict[from_name] = mapping_dict.get(from_name, set()) | set(column_list)
        elif 'join' in x:
            if x.count('join') == 1 and 'union' not in x:
                start_index = x.find("select ") + len("select ")
                end_index = x.find("from")
                column = x[start_index:end_index].replace('  ', '')
                column_list = [item for item in column.split('\n') if item != '']
                from_name = x[end_index + len('from '):]
                sources_name = [z for z in from_name.replace('  ','').split('\n') if 'z2' in z or 'z9' in z]
                for x in sources_name:
                    source_name_letter[x.split(' ')[-1]] = x.split(' ')[-2]     
                on_name = [z for z in from_name.replace('  ','').split('\n') if 'on' in z or 'and' in z]
                source_letter = [d.split('as ')[1] if 'as' in d else d.split(' ')[-1] for d in sources_name ]
                column_used = []
                for p in source_letter:
                    column_used.extend([f"{p}." + u.split('.')[1].split(',')[0].split(' ')[0] for u in column_list if f"{p}." in u])
                    column_used.extend([f"{p}." + f.split(f"{p}.")[1].split(',')[0].split(' ')[0] for f in on_name if f"{p}." in f])
                end_index_from = from_name.find('\n')
                if 'as' in sources_name:
                    from_name = sources_name[0].split(' join ')[1].split(' as ')[0]
                else:
                    from_name = sources_name[0].split(' join ')[1].split(' ')[0]
                
                mapping_dict[from_name] = mapping_dict.get(from_name, set()) | set(column_used)
                for k, v in mapping_dict.items():  
                    for elem in list(mapping_dict[k]):
                        if '.' in elem:  
                            print(elem)  # Print the element containing a dot  
                            # Check if the key does not match the corresponding value in source_name_letter  
                            if k != source_name_letter.get(elem.split('.')[0], None):  
                                mapping_dict[k].discard(elem)  # Discard elem if the condition is true  
                            else:  
                                mapping_dict[k].discard(elem)  # Discard elem again (this seems redundant)  
                                # Split the element by dot and add the second part to the set  
                                second_part = elem.split('.')[1]  
                                mapping_dict[k] = mapping_dict.get(k, set()).union({second_part})  
            else:
                if 'union' in x:
                    x_list = x.split('union')
                    for y in x_list:
                        if 'z2' in y or 'z9' in y:
                            start_index = y.find("select ") + len("select ")
                            end_index = y.find("from")
                            column = y[start_index:end_index].replace('  ', '')
                            column_list = [item.split('--')[0].strip().split(' as ')[0].split(',')[0].split('(')[-1].split(')')[0] for item in column.split('\n') if item.split('--')[0].strip() != '']
                            from_name = y[end_index + len('from '):]
                            if 'join' in from_name:
                                sources_name = [z for z in from_name.replace('  ','').split('\n') if 'z2' in z or 'z9' in z]
                                on_name = [z for z in from_name.split('\n') if 'on' in z or 'and' in z]
                                source_letter = [d.split('as ')[1] if 'as' in d else d.split(' ')[-1] for d in sources_name ]
                                source_name_letter = dict()   
                                for x in sources_name:
                                    source_name_letter[x.split(' ')[-1]] = x.split(' ')[-2]                  
                                column_used = []
                                for p in source_letter:
                                    column_used.extend([f"{p}." + u.split('.')[1].split(',')[0].split(' ')[0] for u in column_list if f"{p}." in u])
                                    column_used.extend([f"{p}." + f.split(f"{p}.")[1].split(',')[0].split(' ')[0] for f in on_name if f"{p}." in f])
                                end_index_from = from_name.find('\n')
                                
                                if 'as' in sources_name:
                                    from_name_join = sources_name[0].split(' join ')[1].split(' as ')[0]
                                elif len(sources_name) == 2:
                                    from_name_join = sources_name[1].split(' join ')[1].split(' ')[0]
                                else:
                                    from_name_join = sources_name[0].split(' ')[0]

                                mapping_dict[from_name_join] = mapping_dict.get(from_name_join, set()) | set(column_used)
                                for k, v in mapping_dict.items():  
                                    for elem in list(mapping_dict[k]):
                                        if '.' in elem: 
                                            # Check if the key does not match the corresponding value in source_name_letter  
                                            if k != source_name_letter.get(elem.split('.')[0], None):  
                                                mapping_dict[k].discard(elem)  # Discard elem if the condition is true  
                                            else:  
                                                mapping_dict[k].discard(elem)  # Discard elem again (this seems redundant)  
                                                # Split the element by dot and add the second part to the set  
                                                second_part = elem.split('.')[1]  
                                                mapping_dict[k] = mapping_dict.get(k, set()).union({second_part})  
                                    
                            end_index_from = from_name.find('\n')
                            from_name = from_name[:end_index_from]
                            source_letter = [d.split('as ')[1] if 'as' in d else d.split(' ')[-1] for d in sources_name ]
                            column_used = []
                            for p in source_letter:
                                column_used.extend([u.split('.')[1].split(',')[0].split(' ')[0] for u in column_list if f"{p}." in u])
                                column_used.extend([f.split(f"{p}.")[1].split(',')[0].split(' ')[0] for f in on_name if f"{p}." in f])
                            mapping_dict[from_name.split(' ')[0]] = mapping_dict.get(from_name.split(' ')[0], set()) | set(column_used)

        
                        
from pyspark.sql.functions import explode, array_contains
mapping_df = spark.createDataFrame([(k, list(v)) for k, v in mapping_dict.items()], ["table_name", "columns"])
display(mapping_df)

            
